In [15]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler



In [16]:
def load_tweets(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        tweets = file.readlines()
    return [tweet.strip() for tweet in tweets]

def load_labels(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        labels = file.readlines()
    return [label.strip() for label in labels]

In [17]:
tweets_file_path = 'train_text.txt'
emojis_file_path = 'train_labels.txt'

# Load tweets and labels
X = load_tweets(tweets_file_path)
y = load_labels(emojis_file_path)

# Assuming 'X' contains your preprocessed tweet text and 'y' contains corresponding emojis
X_train, X_test_locked, y_train, y_test_locked = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2,stratify=y_train, random_state=42)

# Apply oversampling to the training set
#oversampler = RandomOverSampler(random_state=42)
#X_train, y_train = oversampler.fit_resample(X_train, y_train)

In [18]:
# Convert text data to numerical features using TF-IDF
tfidf_vectorizer = TfidfVectorizer(strip_accents='ascii', max_features=5000, sublinear_tf=True)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test_locked)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# Train SVC
svm_model = SVC(kernel='rbf',gamma=1 , class_weight='balanced')
svm_model.fit(X_train_tfidf, y_train)

In [ ]:
# Make predictions on the test set
y_pred = svm_model.predict(X_test_tfidf)

In [ ]:
from sklearn.metrics import f1_score
# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
f1=f1_score(y_test, y_pred, average='weighted')
print(f1)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
# Compute the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Visualize the confusion matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix (Test Set)')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)